<span style="color:#1E90FF; font-weight:bold">**Introduction**

<span style="color:#1E90FF; font-weight:bold">The aim of this exercise is to code an execution algorithmic strategy that tries to achieve an execution cost close to the VWAP market benchmark (Volume Weighted Average Price). It follows the optimal execution strategy to track VWAP discussed in Busseti & Boyd (2015), see the notes of the course for details.

<span style="color:#1E90FF; font-weight:bold">Delivery: Jupyter notebook (Python) with all the compulsory parts of the exercise and results</span>


<span style="color:#1E90FF; font-weight:bold">**Data reading and preparation**

<span style="color:#1E90FF; font-weight:bold">Import the csv file “volume_price_2014_18.csv” required for the exercise. It contains four columns: date, time, volume and price. Load it into a Pandas dataframe. The data corresponds to market volumes and volume weighted average prices for intervals of one minute from 2014 to 2018. Perform an exploratory analysis in order to detect potential issues (hint: days with zero volume). Discard any anomaly you think it can affect the quality of the analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [76]:

class ExecutionAlgorithm:
    def __init__(self, file_path, train_end_date):
        """
        Inicializa la clase ExecutionAlgorithm con la ruta del archivo de datos y la fecha límite para dividir el conjunto de datos.
        
        Args:
            file_path (str): Ruta del archivo CSV con los datos de volumen y precio.
            train_end_date (str): Fecha límite para dividir los datos en entrenamiento y prueba.
        """
        self.file_path = file_path
        self.train_end_date = pd.Timestamp(train_end_date)
        self.data_df = None
        self.filtered_df = None
        self.resampled_df = None
        self.train_df = None
        self.test_df = None
        self.static_volume_curve = None
        self.normalized_volume_curve_result = None

    def load_data(self):
        """
        Carga los datos desde el archivo CSV y maneja excepciones si el archivo no existe o está vacío.
        """
        try:
            # Intentar cargar el archivo CSV
            self.data_df = pd.read_csv(self.file_path)
        except FileNotFoundError:
            # Manejar el caso en el que el archivo no se encuentra
            print(f"Error: El archivo {self.file_path} no fue encontrado.")
        except pd.errors.EmptyDataError:
            # Manejar el caso en el que el archivo está vacío
            print(f"Error: El archivo {self.file_path} está vacío.")
        except Exception as e:
            # Manejar cualquier otra excepción que ocurra durante la carga del archivo
            print(f"Error al leer el archivo: {e}")

    def preprocess_data(self):
        """
        Filtra los datos y realiza cálculos previos al análisis, como el cálculo del VWAP.
        - Elimina valores faltantes.
        - Filtra filas con volumen mayor a 0.
        - Calcula la columna 'vol_price' para el cálculo del VWAP.
        - Convierte las columnas de tiempo y fecha a sus formatos adecuados.
        - Agrupa los datos en intervalos de 5 minutos y calcula el VWAP.
        """
        if self.data_df is not None:
            # Manejar valores faltantes eliminando filas con datos faltantes
            self.data_df = self.data_df.dropna()
            # Filtrar filas con volumen mayor a 0
            self.filtered_df = self.data_df[self.data_df['volume'] > 0].copy()
            # Crear la columna vol_price para el VWAP
            self.filtered_df['vol_price'] = self.filtered_df['volume'] * self.filtered_df['price']
            # Convertir las columnas de tiempo y fecha a sus formatos adecuados
            self.filtered_df['time'] = pd.to_timedelta(self.filtered_df['time'])
            self.filtered_df['date'] = pd.to_datetime(self.filtered_df['date'])
            # Crear columna 'time_5min' para redondear a intervalos de 5 minutos
            self.filtered_df['time_5min'] = self.filtered_df['time'].dt.floor('5min')
            # Agrupar por 'date' y 'time_5min' para sumar el volumen y vol_price
            self.resampled_df = self.filtered_df.groupby(['date', 'time_5min']).agg({
                'volume': 'sum',
                'vol_price': 'sum'
            }).reset_index()
            # Calcular el VWAP
            self.resampled_df['vwap'] = self.resampled_df['vol_price'] / self.resampled_df['volume']
            # Eliminar filas con volumen igual a 0 después del resampleo
            self.resampled_df = self.resampled_df[self.resampled_df['volume'] > 0]
            # Eliminar la columna vol_price ya que ya no es necesaria
            self.resampled_df = self.resampled_df.drop(columns=['vol_price'])
        else:
            print("Error: No se han cargado datos para procesar.")

    def split_data(self):
        """
        Divide los datos en conjuntos de entrenamiento y prueba según la fecha límite especificada.
        """
        if self.resampled_df is not None:
            # Dividir los datos en entrenamiento y prueba utilizando la fecha límite
            self.train_df = self.resampled_df[self.resampled_df['date'] < self.train_end_date].copy()
            self.test_df = self.resampled_df[self.resampled_df['date'] >= self.train_end_date].copy()
            # Mostrar el número de filas en cada conjunto
            print(f"El dataset de entrenamiento tiene: {len(self.train_df)} filas y el de testeo tiene: {len(self.test_df)} filas")
        else:
            print("Error: No se han procesado datos para dividir.")

    def evaluate_model(self):
        """
        Evaluación básica del conjunto de datos dividido, mostrando las primeras filas del conjunto de entrenamiento y prueba.
        """
        if self.train_df is not None and self.test_df is not None:
            # Mostrar la primera fila del conjunto de entrenamiento
            print("Primera fila de train_df:")
            print(self.train_df.head(1))
            # Mostrar la primera fila del conjunto de prueba
            print("\nPrimera fila de test_df:")
            print(self.test_df.head(1))
        else:
            print("Error: No se han dividido los datos para evaluar.")

    def plot_training_volume_histogram(self):
        """
        Para cada día en el conjunto de entrenamiento, calcula el volumen total del mercado del día y grafica el histograma.
        """
        if self.train_df is not None:
            # Calcular el volumen total por día en el conjunto de entrenamiento
            daily_volume = self.train_df.groupby('date')['volume'].sum()
            # Crear el histograma interactivo en Plotly
            fig = go.Figure()
            # Agregar el histograma de 'daily_volume'
            fig.add_trace(go.Histogram(
                x=daily_volume,
                nbinsx=30,
                marker=dict(color='skyblue', line=dict(color='black', width=1)),
                opacity=0.7
            ))
            # Configuración del layout
            fig.update_layout(
                title='Histograma del Volumen Diario Total en el Conjunto de Entrenamiento',
                xaxis_title='Volumen Diario Total',
                yaxis_title='Frecuencia',
                template='plotly_dark',
                bargap=0.1  # Ajuste del espacio entre barras
            )
            # Mostrar la figura
            fig.show()
        else:
            print("Error: No se han dividido los datos para graficar el volumen del entrenamiento.")

    def calculate_average_daily_volume(self):
        """
        Calcula el volumen diario promedio (ADV) como media y también como mediana sobre los volúmenes diarios.
        Compara la diferencia entre la media y la mediana, y discute las ventajas y desventajas de cada una.
        """
        if self.train_df is not None:
            # Calcular el volumen total por día en el conjunto de entrenamiento
            daily_volume = self.train_df.groupby('date')['volume'].sum()
            # Calcular la media y la mediana del volumen diario
            mean_adv = daily_volume.mean()
            median_adv = daily_volume.median()
            # Mostrar los resultados
            print(f"Volumen Diario Promedio (Media): {mean_adv:.2f}")
            print(f"Volumen Diario Promedio (Mediana): {median_adv:.2f}")
            print(f"Diferencia entre Media y Mediana: {abs(mean_adv - median_adv):.2f}")
            
            # Explicar ventajas y desventajas
            print("\nVentajas de usar la Media:")
            print("- Captura todos los valores, incluyendo valores extremos.")
            print("- Es útil cuando los datos tienen una distribución simétrica.")
            print("\nDesventajas de usar la Media:")
            print("- Es sensible a valores atípicos (outliers), lo que puede distorsionar el promedio.")
            
            print("\nVentajas de usar la Mediana:")
            print("- No se ve afectada por valores atípicos, lo que la hace más representativa en distribuciones asimétricas.")
            print("- Es más robusta cuando los datos tienen valores extremos.")
            
            print("\nDesventajas de usar la Mediana:")
            print("- No captura la magnitud de los valores extremos.")
            print("- Puede ser menos precisa si los valores están uniformemente distribuidos sin grandes outliers.")
        else:
            print("Error: No se han dividido los datos para calcular el volumen diario promedio.")

    def analyze_days_of_week(self):
        """
        Separa el conjunto de entrenamiento por días de la semana (lunes, martes, ... viernes) y realiza el análisis para cada día.
        Grafica el volumen promedio para cada día de la semana y compara los resultados con los resultados globales.
        """
        if self.train_df is not None:
            # Añadir una columna con el día de la semana (0=Lunes, 6=Domingo)
            self.train_df['day_of_week'] = self.train_df['date'].dt.weekday
            
            # Diccionario para mapear los números de los días de la semana a sus nombres
            weekday_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday'}
            
            # Inicializar el diccionario para almacenar los resultados
            weekday_vad_stats = {}
            
            # Calcular el volumen diario promedio (media y mediana) para cada día de la semana
            for weekday, weekday_name in weekday_map.items():
                # Filtrar el DataFrame para obtener solo los datos del día de la semana específico
                daily_volume_train = self.train_df[self.train_df['day_of_week'] == weekday].groupby('date')['volume'].sum()
                
                # Calcular la media y la mediana del volumen diario
                vad_mean = daily_volume_train.mean()
                vad_median = daily_volume_train.median()
                
                # Guardar los resultados en el diccionario
                weekday_vad_stats[weekday_name] = {'mean': vad_mean, 'median': vad_median}
            
            # Mostrar los resultados para cada día de la semana
            for day, stats in weekday_vad_stats.items():
                print(f"{day}:")
                print(f"  Volumen Medio Diario (VAD) - Media: {stats['mean']:.2f}")
                print(f"  Volumen Medio Diario (VAD) - Mediana: {stats['median']:.2f}\n")
            
            # Datos para el gráfico
            dias_semana = list(weekday_vad_stats.keys())
            volumen_medio = [stats['mean'] for stats in weekday_vad_stats.values()]
            # Crear el gráfico de barras en Plotly
            fig = go.Figure(go.Bar(
                x=dias_semana,
                y=volumen_medio,
                marker=dict(color='skyblue', line=dict(color='black', width=1)),
                opacity=0.7
            ))
            # Configuración del layout
            fig.update_layout(
                title='Volumen Medio Diario por Día de la Semana en el Conjunto de Entrenamiento',
                xaxis_title='Día de la Semana',
                yaxis_title='Volumen Medio Diario (Media)',
                template='plotly_dark',
                bargap=0.2  # Espacio entre las barras
            )
            # Mostrar la figura
            fig.show()
        else:
            print("Error: No se han dividido los datos para analizar por día de la semana.")

    def static_volume_prediction(self):
        """
        Predicción estática del volumen: calcula la mediana de los volúmenes para cada intervalo de cinco minutos usando
        los datos del mismo intervalo a lo largo de los días del conjunto de entrenamiento. Obtiene una curva de volumen
        mediano con tantos puntos como intervalos de cinco minutos en un día.
        """
        if self.train_df is not None:
            # Agrupar por el intervalo de 5 minutos y calcular la mediana del volumen para cada intervalo
            median_volume_curve = self.train_df.groupby('time_5min')['volume'].median()
            
            # Guardar la curva de volumen mediano para uso posterior
            self.static_volume_curve = median_volume_curve
            
            # Convertir los índices de 'time_5min' a un formato de hora más claro usando timedelta
            time_labels = [str(timedelta)[7:12] for timedelta in median_volume_curve.index]
            
            # Crear la curva de volumen mediano en Plotly
            fig = go.Figure(go.Scatter(
                x=time_labels,
                y=median_volume_curve.values,
                mode='lines',
                line=dict(color='blue', width=2)
            ))
            # Configuración del layout
            fig.update_layout(
                title='Curva de Volumen Mediano por Intervalos de 5 Minutos',
                xaxis_title='Hora de Inicio del Intervalo de 5 Minutos',
                yaxis_title='Volumen Mediano',
                template='plotly_dark'
            )
            # Ajuste de los ticks del eje x
            fig.update_xaxes(tickvals=np.arange(0, len(time_labels), len(time_labels)//10),
                            ticktext=np.array(time_labels)[::len(time_labels)//10], tickangle=45)

            # Mostrar la figura
            fig.show()

        else:
            print("Error: No se han dividido los datos para calcular la predicción de volumen estática.")

    def normalized_volume_curve(self):
        """
        Calcula la curva de volumen normalizado dividiendo cada volumen mediano por el volumen total.
        La curva resultante debe sumar 1. Luego, grafica la curva normalizada.
        """
        if self.static_volume_curve is not None:
            # Calcular el volumen total de la curva estática
            total_volume = self.static_volume_curve.sum()
            
            # Calcular la curva de volumen normalizado
            normalized_curve = self.static_volume_curve / total_volume
            
            # Guardar la curva de volumen normalizado
            self.normalized_volume_curve_result = normalized_curve
            
            # Convertir los índices de 'time_5min' a un formato de hora más claro usando timedelta
            time_labels = [str(timedelta)[7:12] for timedelta in normalized_curve.index]
            
            # Graficar la curva de volumen normalizado en Plotly
            fig = go.Figure(go.Scatter(
                x=time_labels,
                y=normalized_curve.values,
                mode='lines',
                line=dict(color='green', width=2)
            ))
            # Configuración del layout
            fig.update_layout(
                title='Curva de Volumen Normalizado por Intervalos de 5 Minutos',
                xaxis_title='Hora de Inicio del Intervalo de 5 Minutos',
                yaxis_title='Volumen Normalizado',
                template='plotly_dark'
            )
            # Ajuste de los ticks del eje x
            fig.update_xaxes(
                tickvals=np.arange(0, len(time_labels), len(time_labels) // 10),
                ticktext=np.array(time_labels)[::len(time_labels) // 10],
                tickangle=45
            )
            # Mostrar la figura
            fig.show()

        else:
            print("Error: No se ha generado la curva de volumen estática para calcular la curva de volumen normalizado.")

    def static_volume_by_weekday(self):
        """
        Repite el análisis de volumen estático discriminando entre los días de la semana para obtener cinco curvas
        de volumen estático diferentes, una para cada día de la semana. Grafica las curvas estáticas para cada día de la semana.
        """
        if self.train_df is not None:
            # Añadir columna con el día de la semana
            self.train_df['day_of_week'] = self.train_df['date'].dt.day_name()
            weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
            
            # Crear la figura
            fig = go.Figure()

            # Graficar la curva de volumen mediano para cada día de la semana
            for day in weekday_names:
                # Filtrar los datos para el día de la semana específico
                day_df = self.train_df[self.train_df['day_of_week'] == day]
                # Agrupar por el intervalo de 5 minutos y calcular la mediana del volumen para cada intervalo
                median_volume_curve = day_df.groupby('time_5min')['volume'].median()
                # Convertir los índices de 'time_5min' a un formato de hora más claro
                time_labels = [str(timedelta)[7:12] for timedelta in median_volume_curve.index]
                
                # Añadir la curva de volumen mediano para el día de la semana
                fig.add_trace(go.Scatter(
                    x=time_labels,
                    y=median_volume_curve.values,
                    mode='lines',
                    name=f'Volumen Mediano - {day}',
                    line=dict(width=2)
                ))

            # Configuración del layout
            fig.update_layout(
                title='Curvas de Volumen Mediano por Día de la Semana',
                xaxis_title='Hora de Inicio del Intervalo de 5 Minutos',
                yaxis_title='Volumen Mediano',
                template='plotly_dark',
                showlegend=True
            )

            # Ajuste de los ticks del eje x para que no estén tan juntos
            fig.update_xaxes(
                tickvals=np.arange(0, len(time_labels), max(1, len(time_labels) // 5)),
                tickangle=45
            )

            # Mostrar la figura
            fig.show()

        else:
            print("Error: No se han dividido los datos para analizar la curva de volumen estático por días de la semana.")

    def evaluate_static_volume_predictor(self):
        """
        Evalúa la calidad del predictor de volumen estático en el conjunto de prueba usando RMSE como métrica de calidad.
        Normaliza cada perfil diario de volumen del conjunto de prueba y compara la predicción para cada intervalo de 5 minutos
        con el valor real.
        """
        if self.test_df is not None and self.normalized_volume_curve_result is not None:
            # Lista para almacenar los errores por día
            rmse_list = []
            
            # Agrupar los datos de prueba por cada día
            for date, group in self.test_df.groupby('date'):
                # Normalizar el perfil de volumen diario dividiendo por el volumen total del día
                daily_volume_profile = group.groupby('time_5min')['volume'].sum()
                total_daily_volume = daily_volume_profile.sum()
                normalized_daily_profile = daily_volume_profile / total_daily_volume
                
                # Alinear los índices para que coincidan con los de la curva de volumen normalizado
                aligned_profile = normalized_daily_profile.reindex(self.normalized_volume_curve_result.index, fill_value=0)
                
                # Calcular el error cuadrático medio entre la predicción estática y el perfil diario normalizado
                rmse = np.sqrt(mean_squared_error(self.normalized_volume_curve_result, aligned_profile))
                rmse_list.append(rmse)
                
            # Calcular el RMSE promedio en todos los días del conjunto de prueba
            average_rmse = np.mean(rmse_list)
            print(f"RMSE Promedio del Predictor de Volumen Estático en el Conjunto de Prueba: {average_rmse:.4f}")
        else:
            print("Error: No se han dividido los datos de prueba o no se ha generado la curva de volumen normalizado para evaluar el predictor de volumen estático.")

    def evaluate_uniform_volume_predictor(self):
        """
        Evalúa la calidad del predictor de volumen uniforme en el conjunto de prueba usando RMSE como métrica de calidad.
        En lugar de utilizar la curva estática, se asume una predicción uniforme, es decir, el mismo porcentaje para cada intervalo.
        """
        if self.test_df is not None:
            # Crear un predictor uniforme: dividir equitativamente el volumen en todos los intervalos
            num_intervals = len(self.normalized_volume_curve_result) if self.normalized_volume_curve_result is not None else 288  # 288 intervalos de 5 minutos en un día
            uniform_prediction = np.ones(num_intervals) / num_intervals
            
            # Lista para almacenar los errores por día
            rmse_list = []
            
            # Agrupar los datos de prueba por cada día
            for date, group in self.test_df.groupby('date'):
                # Normalizar el perfil de volumen diario dividiendo por el volumen total del día
                daily_volume_profile = group.groupby('time_5min')['volume'].sum()
                total_daily_volume = daily_volume_profile.sum()
                normalized_daily_profile = daily_volume_profile / total_daily_volume
                
                # Alinear los índices para que coincidan con los del predictor uniforme
                aligned_profile = normalized_daily_profile.reindex(self.normalized_volume_curve_result.index, fill_value=0)
                
                # Calcular el error cuadrático medio entre la predicción uniforme y el perfil diario normalizado
                rmse = np.sqrt(mean_squared_error(uniform_prediction, aligned_profile))
                rmse_list.append(rmse)
                
            # Calcular el RMSE promedio en todos los días del conjunto de prueba
            average_rmse = np.mean(rmse_list)
            print(f"RMSE Promedio del Predictor Uniforme en el Conjunto de Prueba: {average_rmse:.4f}")
        else:
            print("Error: No se han dividido los datos de prueba para evaluar el predictor de volumen uniforme.")



<span style="color:#1E90FF; font-weight:bold">Resample the data to intervals of five minutes that will make volume prediction an easier task. For each five minutes interval, the volume is the sum of volumes for each one minute interval, and the price is the volume weighted average price of one minute intervals. [Check: the total volume per day after resampling cannot change with respect to original one minute intervals]. The result should be a dataset with a row per day and a column per 5 minute interval (bin). For simplicity to work with the backtesting, generate columns time (e.g. 9:00:00), date (2014-01-02), volume, vwap

In [77]:
# Definir la ruta del archivo y la fecha de corte para el conjunto de entrenamiento
file_path = "volume_price_2014_18.csv"
train_end_date = '2017-01-02'
# Crear una instancia de la clase ExecutionAlgorithm
algo = ExecutionAlgorithm(file_path, train_end_date)
# Cargar los datos desde el archivo CSV
algo.load_data()
# Preprocesar los datos cargados
algo.preprocess_data()

<span style="color:#1E90FF; font-weight:bold">Divide the dataset in two parts: training and test. Take the first three years for training and the last year for test. Do not use the test set in any training stage!! It will be used exclusively to assess the performance of your model and execution algorithm.

In [78]:
# Dividir los datos en conjuntos de entrenamiento y prueba
algo.split_data()

El dataset de entrenamiento tiene: 75210 filas y el de testeo tiene: 31612 filas


<span style="color:#1E90FF; font-weight:bold">For each day in the training set, calculate the total market volume of the day, and plot the histogram

In [79]:
# Graficar el histograma del volumen diario total en el conjunto de entrenamiento
algo.plot_training_volume_histogram()

<span style="color:#1E90FF; font-weight:bold">Calculate the average daily volume (ADV) as a mean and also as a median over daily volumes. How much do they differ? What are the advantages and disadvantages of using median vs mean to get an “average” estimation

In [80]:
# Calcular el volumen diario promedio (ADV) y discutir la diferencia entre media y mediana
algo.calculate_average_daily_volume()

Volumen Diario Promedio (Media): 1261181.74
Volumen Diario Promedio (Mediana): 1136875.00
Diferencia entre Media y Mediana: 124306.74

Ventajas de usar la Media:
- Captura todos los valores, incluyendo valores extremos.
- Es útil cuando los datos tienen una distribución simétrica.

Desventajas de usar la Media:
- Es sensible a valores atípicos (outliers), lo que puede distorsionar el promedio.

Ventajas de usar la Mediana:
- No se ve afectada por valores atípicos, lo que la hace más representativa en distribuciones asimétricas.
- Es más robusta cuando los datos tienen valores extremos.

Desventajas de usar la Mediana:
- No captura la magnitud de los valores extremos.
- Puede ser menos precisa si los valores están uniformemente distribuidos sin grandes outliers.


La media es útil cuando los datos están distribuidos de manera simétrica y no contienen muchos valores atípicos, pero puede ser engañosa si hay días con volúmenes muy extremos.
La mediana es una opción más robusta para estimar el volumen "típico" en casos donde la distribución es asimétrica o contiene valores extremos. Es menos susceptible a ser influenciada por outliers.

<span style="color:#1E90FF; font-weight:bold">Separate the train set in days of the week (Mondays, Tuesdays… Fridays) and redo the analysis for each day of the week. Do you see any meaningful difference between these results and the global results? Do you see any pattern across days of the week?</span>


In [81]:
# Analizar los volúmenes por días de la semana
algo.analyze_days_of_week()

Monday:
  Volumen Medio Diario (VAD) - Media: 1165484.28
  Volumen Medio Diario (VAD) - Mediana: 1043696.00

Tuesday:
  Volumen Medio Diario (VAD) - Media: 1197613.62
  Volumen Medio Diario (VAD) - Mediana: 1116151.00

Wednesday:
  Volumen Medio Diario (VAD) - Media: 1174438.82
  Volumen Medio Diario (VAD) - Mediana: 1089270.50

Thursday:
  Volumen Medio Diario (VAD) - Media: 1339089.11
  Volumen Medio Diario (VAD) - Mediana: 1194949.00

Friday:
  Volumen Medio Diario (VAD) - Media: 1433041.59
  Volumen Medio Diario (VAD) - Mediana: 1268933.00



<span style="color:#1E90FF; font-weight:bold">[Optional] Can you think about other kind of special days where a specific volume curve should be constructed? Which kind of volume profiles have these days?

In [82]:
# Días de anuncions de política monetarias y económicas
# Días de publicación de datos económicos clave (empleo, inflación, PIB)
# Vencimiento de contratos de futuros y opciones (conocido como "quadruple witching")
# Días de rebalanceo de índices (como el S&P 500)
# Días previos y posteriores a festivos

<span style="color:#1E90FF; font-weight:bold">**Static volume prediction**

<span style="color:#1E90FF; font-weight:bold">For each day (don’t differentiate here between days of the week) calculate the median of volumes for each interval of five minutes using the data from the same interval across days of the training set. You have to get a single median volume curve with as many points as five minute intervals in a day.

In [83]:
# Realizar la predicción estática del volumen
algo.static_volume_prediction()


<span style="color:#1E90FF; font-weight:bold">Get the total volume of this curve, and generate a normalized volume curve by dividing each median volume per interval by the total volume. The resulting normalized curve has to sum to 1. This is a baseline static volume predictor. Plot the static curve. You should see the characteristic "U" shape of market volumes intraday, where most of the the trading happens at the beginning and the end of the trading session. Or more accurately, a "J" shape, with more trading activity at the end of the trading session.

In [84]:
# Realizar la predicción de volumen normalizado
algo.normalized_volume_curve()

<span style="color:#1E90FF; font-weight:bold">Repeat the same analysis but discriminating across days of the week so you get five different static volume curves, one per day of the week. You can also optionally consider other special days where you have detected particular volume profiles. Plot the static curves for each day of the week (and special days).

In [85]:
# Realizar la predicción de volumen estático discriminando por días de la semana
algo.static_volume_by_weekday()

<span style="color:#1E90FF; font-weight:bold">Evaluate the quality of this baseline static volume predictor in the test set. For that, you need to normalize to one each of the daily volume profiles of the test set, and then compare the prediction for each 5 minute interval with the actual value. Use RMSE as the quality metric.

In [12]:
# Evaluar la calidad del predictor de volumen estático en el conjunto de prueba
algo.evaluate_static_volume_predictor()

RMSE Promedio del Predictor de Volumen Estático en el Conjunto de Prueba: 0.0066


<span style="color:#1E90FF; font-weight:bold">This error as it is does not tell us much. We need to compare with some benchmark to make sense of it. Use as a benchmark a uniform prediction, i.e., the same percentage for each bin

In [13]:
# Evaluar la calidad del predictor de volumen uniforme en el conjunto de prueba
algo.evaluate_uniform_volume_predictor()

RMSE Promedio del Predictor Uniforme en el Conjunto de Prueba: 0.0076


<span style="color:#1E90FF; font-weight:bold">Improve your prediction: instead of using a static separation train - test, using a moving average to compute the static curves. You can play with the length N of the window (before it was 3 years, maybe shorter works better?). In this case, for each day of the test set, 1) generate the static prediction using the N previous days, 2) compare this prediction with the actual normalized volume per bin that day and compute the mean square error of the prediction, 3) repeat for every day of the test set, 4) the final RMSE is the square root of the sum of mean square errors per day in the test set. Does the RMSE improve? To simplify the code, wrap it in the following class structure. You will also use it for the trading algorithm later

<span style="color:#1E90FF; font-weight:bold">To instantiate your predictor and train it, use the following code. Assuming historical_df is the dataframe for training:

In [86]:
from sklearn.metrics import mean_squared_error


class StaticBinVolumePredictor:
    def __init__(self, window_size):
        """
        Clase para predecir la curva de volumen utilizando una media móvil para mejorar la separación de entrenamiento y prueba.

        Args:
            window_size (int): Tamaño de la ventana para calcular la media móvil.
        """
        self.window_size = window_size
        self.moving_avg_curve = None

    def train(self, train_df):
        """
        Entrenar el predictor utilizando los datos de volumen con una media móvil.

        Args:
            train_df (DataFrame): Datos de entrenamiento con las columnas ['date', 'time_5min', 'volume', 'vwap', 'day_of_week'].
        """
        # Calcular la media móvil para cada timestamp (promedio de volúmenes anteriores por ventana)
        self.moving_avg_curve = (
            train_df['volume']
            .rolling(window=self.window_size, min_periods=1)
            .mean()
            .groupby(train_df['time_5min'])
            .mean()
        )
        # Normalizar la curva de volumen
        self.moving_avg_curve /= self.moving_avg_curve.sum()

    def predict(self, day_times=None):
        """
        Predecir la curva de volumen estático para un día dado o para toda la serie.

        Args:
            day_times (Index, optional): Índices de tiempo del día a predecir. Si no se proporciona, se predice para todos los tiempos.

        Returns:
            Series: Predicción de la curva de volumen para el día o para toda la serie, con los índices correspondientes.
        """
        if day_times is None:
            return self.moving_avg_curve.fillna(0)
        else:
            return self.moving_avg_curve.reindex(day_times).fillna(0)

    def evaluate(self, test_df):
        """
        Evaluar el rendimiento del predictor comparando las predicciones con los valores reales normalizados.

        Args:
            test_df (DataFrame): Datos de prueba con las columnas ['date', 'time_5min', 'volume', 'vwap'].

        Returns:
            float: RMSE (Raíz del Error Cuadrático Medio) de la predicción.
        """
        mse_list = []
        unique_dates = test_df['date'].unique()
        for date in unique_dates:
            day_df = test_df[test_df['date'] == date]
            actual_curve = day_df.set_index('time_5min')['volume']
            actual_curve /= actual_curve.sum()  # Normalizar la curva de volumen real
            predicted_curve = self.predict(actual_curve.index)
            mse = mean_squared_error(actual_curve, predicted_curve)
            mse_list.append(mse)

        final_rmse = np.sqrt(np.mean(mse_list))

        print(f"RMSE del predictor de volumen estático: {final_rmse}")
        

    def visualize_prediction(self, test_df, date):
        """
        Visualizar la predicción frente a los valores reales normalizados para un día específico.

        Args:
            test_df (DataFrame): Datos de prueba con las columnas ['date', 'time_5min', 'volume', 'vwap'].
            date (str): Fecha específica para la cual se desea visualizar la predicción.
        """
        day_df = test_df[test_df['date'] == date]
        actual_curve = day_df.set_index('time_5min')['volume']
        actual_curve /= actual_curve.sum()  # Normalizar la curva de volumen real
        predicted_curve = self.predict(actual_curve.index)

        # Crear la figura
        fig = go.Figure()

        # Añadir la curva de volumen real normalizado
        fig.add_trace(go.Scatter(
            x=actual_curve.index,
            y=actual_curve.values,
            mode='lines',
            name='Volumen Real Normalizado',
            line=dict(color='blue', width=2)
        ))

        # Añadir la curva de volumen predicho normalizado
        fig.add_trace(go.Scatter(
            x=predicted_curve.index,
            y=predicted_curve.values,
            mode='lines',
            name='Volumen Predicho Normalizado',
            line=dict(color='red', width=2, dash='dash')
        ))

        # Configuración del layout
        fig.update_layout(
            title=f'Predicción de Volumen Normalizado vs Volumen Real para {date}',
            xaxis_title='Hora del Día',
            yaxis_title='Volumen Normalizado',
            template='plotly_dark',
            showlegend=True
        )

        # Ajuste de los ticks del eje x
        fig.update_xaxes(tickangle=45)

        # Mostrar la figura
        fig.show()



# Ejemplo de uso
volume_predictor = StaticBinVolumePredictor(window_size=10)  # Ajustar el hiperparámetro de la ventana

# Asumiendo que 'algo.resampled_df' tiene las columnas ['date', 'time_5min', 'volume', 'vwap']

# Entrenar el predictor
volume_predictor.train(algo.resampled_df)

# Predecir la curva de volumen estático
predicted_curve = volume_predictor.predict()

# Evaluar el rendimiento del predictor
rmse = volume_predictor.evaluate(algo.test_df)

# Visualizar la predicción para una fecha específica
volume_predictor.visualize_prediction(algo.test_df, '2017-01-02')


RMSE del predictor de volumen estático: 0.007438875108908694


<span style="color:#1E90FF; font-weight:bold">Try to improve your prediction using other predictive models. Bear in mind that the target is a normalized vector where all weights sum to one (i.e. akin to multi-class probability problem). Alternatively you can see the problem as a target percentage of volume by using at least one feature: the time bin

In [87]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf
import logging

# Configuración para eliminar mensajes de advertencia
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# Deshabilitar toda salida de TensorFlow
tf.get_logger().setLevel('FATAL')

# Forzar uso de CPU si la GPU está causando problemas
tf.config.set_visible_devices([], 'GPU')

class RNNVolumePredictor:
    def __init__(self, window_size, lstm_units=50, epochs=10, batch_size=64):
        """
        Clase para predecir la curva de volumen utilizando una RNN (LSTM) para mejorar la predicción del volumen.

        Args:
            window_size (int): Tamaño de la ventana para los datos secuenciales.
            lstm_units (int): Número de unidades LSTM en la capa.
            epochs (int): Número de épocas para el entrenamiento.
            batch_size (int): Tamaño del batch para el entrenamiento.
        """
        self.window_size = window_size
        self.lstm_units = lstm_units
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.scaler = MinMaxScaler()

    def train(self, train_df):
        """
        Entrenar el predictor utilizando una RNN (LSTM).

        Args:
            train_df (DataFrame): Datos de entrenamiento con las columnas ['date', 'time_5min', 'volume', 'vwap', 'day_of_week'].
        """
        # Preprocesamiento de los datos
        grouped = train_df.groupby('time_5min')['volume'].mean()
        normalized_volume = self.scaler.fit_transform(grouped.values.reshape(-1, 1)).flatten()

        X, y = [], []
        for i in range(self.window_size, len(normalized_volume)):
            X.append(normalized_volume[i - self.window_size:i])
            y.append(normalized_volume[i])

        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape para LSTM

        # Crear el modelo LSTM
        self.model = Sequential()
        self.model.add(LSTM(units=self.lstm_units, input_shape=(X.shape[1], 1), return_sequences=True))
        self.model.add(LSTM(units=self.lstm_units))
        self.model.add(Dense(1, activation='linear'))
        self.model.compile(optimizer='adam', loss='mean_squared_error')

        # Entrenar el modelo
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)

    def predict(self, test_df):
        """
        Predecir la curva de volumen utilizando el modelo entrenado.

        Args:
            test_df (DataFrame): Datos de prueba con las columnas ['date', 'time_5min', 'volume', 'vwap'].

        Returns:
            Series: Predicción de la curva de volumen normalizada para el día o para toda la serie, con los índices correspondientes.
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado aún.")

        grouped = test_df.groupby('time_5min')['volume'].mean()
        normalized_volume = self.scaler.transform(grouped.values.reshape(-1, 1)).flatten()

        X_pred = []
        for i in range(self.window_size, len(normalized_volume)):
            X_pred.append(normalized_volume[i - self.window_size:i])

        X_pred = np.array(X_pred)
        X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))
        predictions = self.model.predict(X_pred, verbose=0)

        return pd.Series(predictions.flatten(), index=grouped.index[self.window_size:])

    def evaluate(self, test_df):
        """
        Evaluar el rendimiento del predictor comparando las predicciones con los valores reales normalizados.

        Args:
            test_df (DataFrame): Datos de prueba con las columnas ['date', 'time_5min', 'volume', 'vwap'].

        Returns:
            float: RMSE (Raíz del Error Cuadrático Medio) de la predicción.
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado aún.")

        grouped = test_df.groupby('time_5min')['volume'].mean()
        normalized_volume = self.scaler.transform(grouped.values.reshape(-1, 1)).flatten()

        X_test, y_test = [], []
        for i in range(self.window_size, len(normalized_volume)):
            X_test.append(normalized_volume[i - self.window_size:i])
            y_test.append(normalized_volume[i])

        X_test, y_test = np.array(X_test), np.array(y_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        predictions = self.model.predict(X_test, verbose=0).flatten()

        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        print(f"RMSE del predictor de volumen utilizando la RNN: {rmse}")
        return rmse

    def visualize_prediction(self, test_df, date):
        """
        Visualizar la predicción frente a los valores reales normalizados para un día específico.

        Args:
            test_df (DataFrame): Datos de prueba con las columnas ['date', 'time_5min', 'volume', 'vwap'].
            date (str): Fecha específica para la cual se desea visualizar la predicción.
        """
        day_df = test_df[test_df['date'] == date]
        actual_curve = day_df.set_index('time_5min')['volume']
        actual_curve /= actual_curve.sum()  # Normalizar la curva de volumen real
        predicted_curve = self.predict(day_df)
        predicted_curve /= predicted_curve.sum()  # Normalizar la curva de volumen predicho

        
        # Crear la figura en Plotly
        fig = go.Figure()

        # Añadir la curva de volumen real normalizado
        fig.add_trace(go.Scatter(
            x=actual_curve.index,
            y=actual_curve.values,
            mode='lines',
            name='Volumen Real Normalizado',
            line=dict(color='blue', width=2)
        ))

        # Añadir la curva de volumen predicho normalizado
        fig.add_trace(go.Scatter(
            x=predicted_curve.index,
            y=predicted_curve.values,
            mode='lines',
            name='Volumen Predicho Normalizado',
            line=dict(color='red', width=2, dash='dash')
        ))

        # Configuración del layout
        fig.update_layout(
            title=f'Predicción de Volumen Normalizado vs Volumen Real para {date}',
            xaxis_title='Hora del Día',
            yaxis_title='Volumen Normalizado',
            template='plotly_dark',
            showlegend=True
        )

        # Ajuste de los ticks del eje x
        fig.update_xaxes(tickangle=45)

        # Mostrar la figura
        fig.show()


# Ejemplo de uso
volume_predictor = RNNVolumePredictor(window_size=7, lstm_units=50, epochs=10, batch_size=32)  # Ajustar hiperparámetros para mayor estabilidad

# Asumiendo que 'algo.resampled_df' tiene las columnas ['date', 'time_5min', 'volume', 'vwap']

# Entrenar el predictor
volume_predictor.train(algo.resampled_df)

# Evaluar el rendimiento del predictor
rmse = volume_predictor.evaluate(algo.test_df)

# Visualizar la predicción para una fecha específica
volume_predictor.visualize_prediction(algo.test_df, '2017-01-02')

2024-11-13 14:50:17.095553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 14:50:21.757598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-13 14:50:21.759417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/

RMSE del predictor de volumen utilizando la RNN: 0.11871889299184739


<span style="color:#1E90FF; font-weight:bold">**Dynamic volume predictor**

<span style="color:#1E90FF; font-weight:bold">The static volume predictor developed so far does not use the current intraday information to update predictions. This can be a weakness if we observe intraday volume patterns that differ from the historical ones, since the model cannot adapt to them. You will implement a K-nearest neighbours model for dynamic prediction with the following logic:

<span style="color:#1E90FF; font-weight:bold">* The training of the model is simple, we just store all the days of the training set which contain volumes per bin intraday

<span style="color:#1E90FF; font-weight:bold">* For prediction, given the current day with n bins already known, we calculate the Euclidean distance with respect to every curve in the training set (truncated of course to have the same number of bins). We sort the results by this distance and select the K nearest.

<span style="color:#1E90FF; font-weight:bold">* The prediction will be then the median of volume in the next bin using those K nearest neighbours. You will also predict in this model the volume until the end of the day.

<span style="color:#1E90FF; font-weight:bold">* You need to decide how to handle the initilization, when you don't have data from the intraday (or even when you just have a few bins) as to calculate the nearest neighbours.

<span style="color:#1E90FF; font-weight:bold">* The output will be a tuple with the prediction for next bin, and the prediction until the end of the day

<span style="color:#1E90FF; font-weight:bold">I recommend to implement this from scratch in code, but is up to you if you want to adapt any existing implementation of KNN like the one in sklearn.

<span style="color:#1E90FF; font-weight:bold">Wrap your model also in a class structure:

In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import euclidean_distances, mean_squared_error, mean_absolute_error

class dynamicVolumePredictor:
    def __init__(self, k=5):
        """
        Clase para predecir la curva de volumen utilizando un modelo K-Nearest Neighbours (KNN) para predicciones dinámicas.

        Args:
            k (int): Número de vecinos cercanos a considerar para la predicción.
        """
        self.k = k
        self.historical_data = None  # Para almacenar los datos históricos

    def train(self, historical_df):
        """
        Entrenar el modelo almacenando los datos históricos.

        Args:
            historical_df (DataFrame): Datos históricos con columnas ['date', 'time_5min', 'volume'].
        """
        # Almacenar las curvas de volumen diarias para entrenamiento
        self.historical_data = historical_df.pivot(index='date', columns='time_5min', values='volume')

    def predict(self, intraday_df):
        """
        Predecir el volumen para el siguiente intervalo de tiempo y hasta el final del día.

        Args:
            intraday_df (DataFrame): Datos intradía conocidos con columnas ['date', 'time_5min', 'volume'].

        Returns:
            tuple: Predicción del siguiente intervalo y predicción hasta el final del día.
        """
        if self.historical_data is None:
            raise ValueError("El modelo no ha sido entrenado aún con datos históricos.")

        # Obtener la curva parcial del día actual
        current_day = intraday_df['date'].iloc[0]
        current_curve = intraday_df.pivot(index='date', columns='time_5min', values='volume').iloc[0]
        current_curve = current_curve.dropna()  # Eliminar valores no conocidos
        n_bins = len(current_curve)

        # Caso de inicialización: si no hay suficientes datos intradía, usar promedio histórico
        if n_bins == 0:
            avg_volume = self.historical_data.mean(axis=0)
            next_bin_prediction = avg_volume.iloc[0]
            full_day_prediction = avg_volume.sum()
            return next_bin_prediction, full_day_prediction

        # Calcular la distancia euclidiana entre la curva parcial actual y las curvas históricas
        historical_truncated = self.historical_data.iloc[:, :n_bins].dropna()
        distances = euclidean_distances([current_curve], historical_truncated.values)[0]

        # Encontrar los K vecinos más cercanos
        nearest_indices = np.argsort(distances)[:self.k]
        nearest_curves = self.historical_data.iloc[nearest_indices]

        # Predicción para el siguiente intervalo
        next_bin_volumes = nearest_curves.iloc[:, n_bins]
        next_bin_prediction = np.median(next_bin_volumes)

        # Predicción para el resto del día
        remaining_volumes = nearest_curves.iloc[:, n_bins:].sum(axis=1)
        full_day_prediction = np.median(remaining_volumes)

        return next_bin_prediction, full_day_prediction

    def evaluate(self, test_df):
        """
        Evaluar el rendimiento del predictor utilizando datos de prueba.

        Args:
            test_df (DataFrame): Datos de prueba con columnas ['date', 'time_5min', 'volume'].

        Returns:
            dict: Métricas de rendimiento (RMSE, MAE).
        """
        dates = test_df['date'].unique()
        predictions = []
        actuals = []

        for date in dates:
            intraday_df = test_df[test_df['date'] == date].iloc[:10]  # Tomar los primeros 10 registros del día actual
            next_bin_prediction, _ = self.predict(intraday_df)
            actual_volume = test_df[(test_df['date'] == date) & (test_df['time_5min'] == intraday_df['time_5min'].iloc[-1])]['volume'].values[0]
            predictions.append(next_bin_prediction)
            actuals.append(actual_volume)

        rmse = np.sqrt(mean_squared_error(actuals, predictions))
        mae = mean_absolute_error(actuals, predictions)

        return {'RMSE': rmse, 'MAE': mae}

class Order:
    def __init__(self, side, volume):
        self.side = side
        self.volume = volume

class OrderManager:
    def __init__(self):
        self.orders = []

    def submitOrder(self, side, volume):
        order = Order(side, volume)
        self.orders.append(order)

class BacktestingSession:
    def run_backtest(self, historical_df, trading_algo, start_date, end_date=None):
        backtesting_result = []
        full_test_df = historical_df[historical_df["date"] >= start_date]
        if end_date is not None:
            full_test_df = full_test_df[full_test_df["date"] <= end_date]
        test_dates = np.unique(full_test_df["date"])
        # Run backtest for each date between start_date and end_date (if none until last date in historical_df)
        for date in test_dates:
            # Train trading algorithm before the market opens with historical data
            train_df = historical_df[historical_df["date"] < date]
            trading_algo.beforeMarketOpen(train_df)
            # Now start the market session. We shift the dataframe to pass the algo info about previous bin at current time
            test_df = historical_df[historical_df["date"] == date]
            shifted_test_df = test_df.set_index("time_5min").shift(1).reset_index()
            vwap_algo = 0
            volume_algo = 0
            for bin_time in shifted_test_df["time_5min"]:
                # Call onDataEvent and pass the current market information: time (start of next bin), previous volume and vwap
                # It returns order manager object with orders submitted by algo
                order_manager = trading_algo.onDataEvent(shifted_test_df[shifted_test_df["time_5min"] == bin_time])
                vwap_next_bin = float(test_df[test_df["time_5min"] == bin_time]["vwap"].iloc[0])
                total_volume_bin = 0
                for order in order_manager.orders:
                    # we consider potentially several orders, but since we assume all execute at vwap we just net them
                    # if they have different signs
                    total_volume_bin += order.volume * order.side
                volume_algo += np.abs(total_volume_bin)
                vwap_algo += total_volume_bin * vwap_next_bin
            # Calculate the vwap from the algo by normalizing by price executed by volume and calculate market vwap
            vwap_algo = vwap_algo / volume_algo if volume_algo != 0 else 0
            vwap_market = (test_df["volume"] * test_df["vwap"]).sum() / test_df["volume"].sum()
            backtesting_result.append({"date": date, "vwap_algo": vwap_algo, "vwap_market": vwap_market})
        return pd.DataFrame(backtesting_result)

# Ejemplo de uso
# Crear instancia del predictor
dynamic_predictor = dynamicVolumePredictor(k=5)

# Asumiendo que 'algo.resampled_df' tiene las columnas ['date', 'time_5min', 'volume', 'vwap']

# Dividir los datos en entrenamiento y prueba
train_df = algo.resampled_df[algo.resampled_df['date'] < '2017-01-01']
test_df = algo.resampled_df[algo.resampled_df['date'] >= '2017-01-01']

# Entrenar el predictor
dynamic_predictor.train(train_df)

# Evaluar el rendimiento del predictor con los datos de prueba
evaluation_metrics = dynamic_predictor.evaluate(test_df)
# print(f"Métricas de evaluación del modelo dinámico: {evaluation_metrics}")

# Crear una clase de algoritmo de trading que use el predictor dinámico
class TradingAlgorithm:
    def __init__(self, predictor):
        self.predictor = predictor
        self.order_manager = OrderManager()

    def beforeMarketOpen(self, historical_data):
        # Entrenar el predictor antes de que abra el mercado
        self.predictor.train(historical_data)

    def onDataEvent(self, intraday_data):
        # Predecir el siguiente volumen
        next_bin_prediction, _ = self.predictor.predict(intraday_data)
        # Crear una orden basada en la predicción
        side = 1  # Asumimos lado de compra (1) para simplicidad
        volume = next_bin_prediction
        self.order_manager.submitOrder(side, volume)
        return self.order_manager

# Crear instancia del algoritmo de trading
trading_algo = TradingAlgorithm(dynamic_predictor)

# Crear una sesión de backtesting
backtesting_session = BacktestingSession()

# Ejecutar el backtesting
backtesting_result = backtesting_session.run_backtest(algo.resampled_df, trading_algo, start_date='2017-01-01', end_date='2017-12-31')

# Mostrar los resultados del backtesting
print(backtesting_result)

# Calcular el MAE entre vwap_algo y vwap_market
mae_vwap = mean_absolute_error(backtesting_result['vwap_market'], backtesting_result['vwap_algo'])
print(f"MAE entre vwap_algo y vwap_market: {mae_vwap}")

# Crear la figura en Plotly
fig = go.Figure()

# Añadir la curva de VWAP del algoritmo
fig.add_trace(go.Scatter(
    x=backtesting_result['date'],
    y=backtesting_result['vwap_algo'],
    mode='lines',
    name='VWAP Algoritmo',
    line=dict(width=2)
))

# Añadir la curva de VWAP del mercado
fig.add_trace(go.Scatter(
    x=backtesting_result['date'],
    y=backtesting_result['vwap_market'],
    mode='lines',
    name='VWAP Mercado',
    line=dict(width=2, dash='dash')
))

# Configuración del layout
fig.update_layout(
    title='Comparación de VWAP: Algoritmo vs Mercado',
    xaxis_title='Fecha',
    yaxis_title='VWAP',
    template='plotly_dark',
    showlegend=True
)

# Ajuste de los ticks del eje x
fig.update_xaxes(tickangle=45)

# Mostrar la figura
fig.show()



          date   vwap_algo  vwap_market
0   2017-01-02  156.979715   156.856686
1   2017-01-03  159.366235   159.190686
2   2017-01-04  160.371809   160.329800
3   2017-01-05  160.923692   160.868180
4   2017-01-06  160.562283   160.546158
..         ...         ...          ...
238 2017-12-19  200.447950   200.406543
239 2017-12-20  197.998050   197.522090
240 2017-12-21  195.253610   195.313176
241 2017-12-22  195.210285   195.249884
242 2017-12-28  194.166354   194.027319

[243 rows x 3 columns]
MAE entre vwap_algo y vwap_market: 0.08208068823802087


<span style="color:#1E90FF; font-weight:bold">Train your model with training data and evaluate the quality with test data. Compare the metrics with respect to the static model. You should not only consider average predictive quality, also the variance. Also compare the result against baselines, for instance the previous bin volume as a prediction for the next bin, or the previous day total volume (substracting the current executed volume) for the total volume prediction.

<span style="color:#1E90FF; font-weight:bold">[Optional] Try to improve your prediction using other predictive models

<span style="color:#1E90FF; font-weight:bold">**Backtesting environment**

<span style="color:#1E90FF; font-weight:bold">The following code is a simple backtesting environment to run your execution algorithm. Feel free to analyse the code but do not change the implementation

In [66]:
class Order:
    def __init__(self, side, volume):
        self.side = side
        self.volume = volume

class OrderManager():
    def __init__(self):
        self.orders = []

    def submitOrder(self, side, volume):
        order = Order(side, volume)
        self.orders.append(order)

class BacktestingSession:
    def run_backtest(self, historical_df, trading_algo, start_date, end_date = None):
        backtesting_result = pd.DataFrame(columns = ["date", "vwap_algo", "vwap_market"])
        full_test_df = historical_df[historical_df["date"] >= start_date]
        if not end_date is None:
            full_test_df = full_test_df[full_test_df["date"]<= end_date]
        test_dates = np.unique(full_test_df["date"])
        # Run backtest for each date between start_date and end_date (if none until last date in historical_df)
        for date in test_dates:
            # Train trading algorithm before the market opens with historical data
            train_df = historical_df[historical_df["date"] < date]
            trading_algo.beforeMarketOpen(train_df)
            # Now start the market session. We shift the dataframe to pass the algo info about previous bin at current time
            test_df = historical_df[historical_df["date"] == date]
            shifted_test_df = test_df.set_index("time").shift(1).reset_index()
            vwap_algo = 0
            volume_algo = 0
            for bin_time in shifted_test_df["time"]:
                # Call onDataEvent and pass the current market information: time (start of next bin), previous volume and vwap
                # It returns order manager object with orders submitted by algo
                order_manager = myalgo.onDataEvent(shifted_test_df[shifted_test_df["time"] == bin_time])
                vwap_next_bin = float(test_df[test_df["time"] == bin_time]["vwap"])
                total_volume_bin = 0
                for order in order_manager.orders:
                    # we consider potentially several orders, but since we assume all execute at vwap we just net them
                    # if they have different signs
                    total_volume_bin += order.volume * order.side
                volume_algo += np.abs(total_volume_bin)
                vwap_algo += total_volume_bin * vwap_next_bin
            # Calculate the vwap from the algo by normalizing by price executed by volume and calculate market vwap
            vwap_algo = vwap_algo / volume_algo
            vwap_market = (test_df["volume"] * test_df["vwap"]).sum() / test_df["volume"].sum()
            backtesting_result = backtesting_result.append({"date": date, "vwap_algo": vwap_algo, "vwap_market": vwap_market}, ignore_index = True)
        return backtesting_result

<span style="color:#1E90FF; font-weight:bold">**Static VWAP execution algorithm**

<span style="color:#1E90FF; font-weight:bold">Implement a class for a static VWAP execution algorithm with the following structure:
Inputs:

<span style="color:#1E90FF; font-weight:bold">* Amount of shares to execute (Q). Use a check to avoid that the user inputs more than 25% of the ADV calculated above to prevent a huge market impact

<span style="color:#1E90FF; font-weight:bold">* Side of the order: +1 for buy and -1 for sell

<span style="color:#1E90FF; font-weight:bold">* Start time / end time: for this exercise, we will consider that starts at 9:00 and ends at 17:30. We can relax this assumption but then you need to renormalize the static volume curve to the period

<span style="color:#1E90FF; font-weight:bold">* Hyperparameters for the volume predictor, if needed

<span style="color:#1E90FF; font-weight:bold">The logic of the algorithm is the following: every time a time bin of 5 minutes ends, it sends an order to our execution management system (EMS) with a volume given by the static volume predictor:

$q_i = Q E[\frac{v_i}{V}]$

<span style="color:#1E90FF; font-weight:bold">where the term $E[\frac{v_i}{V}]$ is your static volume prediction for the given time bin. The method beforeMarketOpen gets a dataframe with columns: date, time, volume, vwap, day_of_week containing historical volumes and prices per bin in previous days. The method onDataEvent has as input a market data object containing the current time, the executed market volume and VWAP price of the previous bin (e.g. at 9:05:00 reports volume and vwap of bin 9:00:00 - 0:05:00).  It will return a order manager object containing the parent order volume. Our simulation environment will assume that the EMS is able to execute at VWAP price within the next bin the order submitted.

In [103]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

class StaticVWAPAlgorithm:
    def __init__(self, volume, side, max_adv=0.25):
        """
        Initialize the StaticVWAPAlgorithm class.

        Args:
            volume (float): Number of shares to execute (Q).
            side (int): Side of the order: +1 for buy and -1 for sell.
            max_adv (float): Maximum percentage of the Average Daily Volume to execute (default is 25%).
        """
        self.volume = volume
        self.side = side
        self.max_adv = max_adv
        self.static_volume_curve = None
        self.adv = None

    def beforeMarketOpen(self, historical_df):
        """
        Train the model to calculate the static volume curve from historical data.

        Args:
            historical_df (DataFrame): Historical data with columns: date, time_5min, volume, vwap.
        """
        # Ensure the historical_df has the required columns
        if 'time_5min' not in historical_df.columns:
            raise KeyError("The historical DataFrame must contain a 'time_5min' column.")

        # Calculate average daily volume (ADV)
        daily_volume = historical_df.groupby('date')['volume'].sum()
        self.adv = daily_volume.mean()

        # Check if the order volume exceeds the allowed percentage of ADV
        if self.volume > self.max_adv * self.adv:
            raise ValueError("Order volume exceeds 25% of the Average Daily Volume (ADV).")

        # Calculate expected volume curve per time bin
        historical_df['time_bin'] = pd.to_datetime(historical_df['time_5min'], errors='coerce').dt.time
        volume_by_bin = historical_df.groupby('time_bin')['volume'].mean()
        total_volume = volume_by_bin.sum()
        self.static_volume_curve = volume_by_bin / total_volume

    def onDataEvent(self, current_time, market_data):
        """
        Handle a data event and determine the order to send to the EMS.

        Args:
            current_time (datetime.time): The current time of the trading day.
            market_data (dict): Market data containing volume and VWAP for the current bin.

        Returns:
            OrderManager: An object containing the parent order volume for execution.
        """
        time_bin = current_time
        volume_proportion = self.static_volume_curve.get(time_bin, 0)
        child_order_volume = self.volume * volume_proportion

        order_manager = OrderManager()
        order_manager.parent_order_volume = self.side * child_order_volume
        return order_manager

# OrderManager and BacktestingSession would need to be implemented as well.
class OrderManager:
    def __init__(self):
        self.parent_order_volume = 0

# Example of running the backtesting
# backtesting = BacktestingSession()
# volume = 1000
# side = 1
# myalgo = StaticVWAPAlgorithm(volume, side)
# historical_df = algo.resampled_df
# myalgo.beforeMarketOpen(historical_df)
# backtest_results = backtesting.run_backtest(historical_df, myalgo, '2014-01-02')

<span style="color:#1E90FF; font-weight:bold">Now run your execution algorithm over the test set to generate the performance metrics, in our case we will perform daily executions. The backtesting environment returns a dataframe with the VWAP of each execution as well as the market VWAP. Analyse the result:

<span style="color:#1E90FF; font-weight:bold">* For each day of the test set, calculate the percentage error between the trading algorithm VWAP and the market VWAP in basis points:

${\rm pcterror_{vwap}}(bps) = 10000 * \frac{VWAP_{algo} - VWAP_{market}}{VWAP_{market}}$

<span style="color:#1E90FF; font-weight:bold">* Calculate the statistics of this differences in bps over the test set:

<span style="color:#1E90FF; font-weight:bold">* Histogram of the differences

<span style="color:#1E90FF; font-weight:bold">* Mean and standard deviation

<span style="color:#1E90FF; font-weight:bold">* Percentile 1, 5, 95 and 99

<span style="color:#1E90FF; font-weight:bold">* Have a look at the best and worst performance cases, why do you think the algorithm performs so bad/well in these days?

<span style="color:#1E90FF; font-weight:bold">* Compare a buy order versus a sell order, is there any difference in the performance? Would you expect one given the way the trading algorithm is built?
<span style="color:#1E90FF; font-weight:bold">* Imagine you are running a broker-dealer business where you execute orders on behalf of clients. You are going to do this business regularly for buy orders. You guarantee the client the market VWAP for the execution but they have to pay you a fee. You would like to use the histogram from the back-testing to find a fee that on average makes you win money from this business, what would be?

In [105]:
# Example of running the backtesting
backtesting = BacktestingSession()
volume = 1000
side = 1
myalgo = StaticVWAPAlgorithm(volume, side)
historical_df = algo.resampled_df
myalgo.beforeMarketOpen(historical_df)
backtest_results = backtesting.run_backtest(historical_df, myalgo, '2014-01-02')



KeyError: 'time'

In [102]:
historical_df.head()

,date,time_5min,volume,vwap
0,2014-01-02,0 days 09:00:00,24732,131.060911
1,2014-01-02,0 days 09:05:00,22744,131.239107
2,2014-01-02,0 days 09:10:00,23342,131.215468
3,2014-01-02,0 days 09:15:00,24077,131.217174
4,2014-01-02,0 days 09:20:00,24417,131.046016


<span style="color:#1E90FF; font-weight:bold">**Dynamic VWAP execution algorithm**

<span style="color:#1E90FF; font-weight:bold">Let us now use the dynamic volume predictors to build a more reactive execution algorithm. The interface is similar to the static execution algorithm, but now the order placement formula is replaced by the following one:

$q_i = Q \frac{V_{0,i-1} + E_i[v_i]}{E_i[V]} - Q_i$

<span style="color:#1E90FF; font-weight:bold">where again $Q$ is the total volume of our order, and $Q_i$ is the volume executed by the algorithm before next bin $i$. Recall that $E_i[v_i]$ is the dynamic volume prediction for next bin $i$, and $E_i[V]$ the dynamic total market volume prediction (if you built a predictor for the remaining market volume, then you can split it as:  $E_i[V] = V_{0,i-1} + E_i[V_{i,N-1}]$)

In [ ]:
# Your code here
class dynamic_vwap_algorithm():
    def __init__(self, volume, side, start_time, end_time, vol_predictor = None):
        ## implement
        pass

    def beforeMarketOpen(self, historical_df):
        ## train model
        pass

    def onDataEvent(self, intraday_df):
        order_manager = ...
        ## implement
        return order_manager

<span style="color:#1E90FF; font-weight:bold">Redo the backtesting analysis from the previous section and compare the results with the static vwap. Which one is better? Which one would you use if you were running a broker-dealer execution business? Why?